In [ ]:
import logging
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.graph_objs as go
import yfinance as yf
import time
import datetime
pio.renderers.default = 'notebook'


class pfFilter:
    def __init__(self,pf=None):
        self.logger = logging.getLogger(__name__)
        self.alldata_df = None
        self.stocks = self._fetch_stocks(pf) if pf is not None else []
        
    def _fetch_stocks(self,pf):
        try:
            with open(pf,'r') as f:
                return [i.strip() for i in f.readlines()]
        except FileNotFoundError:
            self.logger.warning('Portfolio file not found')
            return []
                    
    def _briefing(self):
        insights = {'52wk_change':'52WeekChange','ps_ttm':'priceToSalesTrailing12Months','pe_ttm':'trailingPE','pe_forward':'forwardPE'}
        briefing_df = pd.DataFrame(columns=insights.keys())
        
        if self.stocks:
            for stock in self.stocks:
                try:
                    time.sleep(0.5)
                    ticker = yf.Ticker(stock)
                    #print(ticker.info.keys())
                    for key,value in insights.items():
                        try:
                            x = ticker.info[value]
                        except Exception as e:
                            x = np.NaN

                        briefing_df.loc[stock,key] = round(x,2)
                        
                except Exception as e:
                    self.logger.warning(e)
        
        briefing_df.sort_values(by='ps_ttm',inplace=True)
        #print(briefing_df)
        return briefing_df 

    def filter(self,return_filter=0.1,pe_dc_filter=0.2):
        filtered_list = []
        
        df = self._briefing()
        if not df.empty:
            mean = df['ps_ttm'].mean()
            for stock in df.index:
                if df.loc[stock,'ps_ttm'] <= mean:
                    fpe = df.loc[stock,'pe_forward']
                    cpe = df.loc[stock,'pe_ttm'] 
                    pe_dc = (cpe / fpe)-1 if fpe > 0 and cpe > 0 else 0
                    if pe_dc >= pe_dc_filter and df.loc[stock,'52wk_change'] >= return_filter:
                        filtered_list.append(stock.strip())
        
        if filtered_list:
            folder = Path('lists')
            if not folder.exists():
                folder.mkdir()
                
            path = Path(f'{folder}/filtered.txt')
            with open(path,'a+') as f:
                content = [i.strip() for i in f.readlines()]

                for stock in filtered_list:
                    if stock not in content:
                        f.seek(0)
                        f.write(f'{stock}\n')
                        self.logger.info(f' appended {stock} to filtered list')
        else:
            self.logger.info('filtered list is empty')
    
    def __call__(self):
        self.filter()


if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)
    obj = pfFilter('lists/industrials.txt')
    obj()